In [34]:
import numpy as np
from scipy.stats import invwishart, invgamma

from tqdm.notebook import tqdm
from scipy.special import gamma 
from scipy.optimize import minimize

In [ ]:
np.random.seed(42)

In [ ]:
true_cov = np.array([[1,0],[0,2]])

def generate_samples(n):
    # size = nx2
    mean = np.array([0,0])
    cov = np.array([[1,0],[0,2]])

    samples = np.random.multivariate_normal(mean,cov,size=n)
    return samples

In [40]:
n10 = generate_samples(10)
n100 = generate_samples(100)
n1000 = generate_samples(1000)

In [39]:
def get_error(samples, estimate):
    return np.linalg.norm(samples - estimate)

## Helper Functions

In [35]:
def gammafn(x,d):
    init = np.pi**(d*(d-1)/4)
    for i in range(1,d+1):
        init *= gamma(x + (1-i)/2)
    return init
def objective_function_voptimal(v,n,d):
    term_1 = v*np.log((v+n)/v)
    term_2 = n*np.log((v+n)/n)
    term_3 = np.log(gammafn(v/2,d)/gammafn((v+n)/2,d))
    return term_1 + term_2 + term_3

## Ques - 1 : MLE

In [37]:
def mle_estimate(samples):
    return samples.T@samples/len(samples)

In [38]:
mle_n10 = mle_estimate(n10)
mle_n100 = mle_estimate(n100)
mle_n1000 = mle_estimate(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {mle_n10[0]} \n \t\t{mle_n10[1]} \t", f"Error : {get_error(true_cov, mle_n10)}" , "\n")
print(f"N100 estimate : {mle_n100[0]} \n \t\t{mle_n100[1]} \t", f"Error : {get_error(true_cov, mle_n100)}", "\n")
print(f"N1000 estimate : {mle_n1000[0]} \n \t\t{mle_n1000[1]} \t", f"Error : {get_error(true_cov, mle_n1000)}" , "\n")  

NameError: name 'get_error' is not defined

### Ques - 2 : Bayesian Est with Conjugate Prior

In [ ]:
def bayesian_estimate_invwishart_cp(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[4,0],[0,5]])
    d = 1
    dof = n + v0
    scale = delta0 + samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

In [ ]:
bayes_invwis_n10 = bayesian_estimate_invwishart_cp(n10)
bayes_invwis_n100 = bayesian_estimate_invwishart_cp(n100)
bayes_invwis_n1000 = bayesian_estimate_invwishart_cp(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {bayes_invwis_n10[0]} \n \t\t{bayes_invwis_n10[1]} \t", f"Error : {get_error(true_cov, bayes_invwis_n10)}" , "\n")
print(f"N100 estimate : {bayes_invwis_n100[0]} \n \t\t{bayes_invwis_n100[1]} \t", f"Error : {get_error(true_cov, bayes_invwis_n100)}", "\n")
print(f"N1000 estimate : {bayes_invwis_n1000[0]} \n \t\t{bayes_invwis_n1000[1]} \t", f"Error : {get_error(true_cov, bayes_invwis_n1000)}" , "\n")  

N10 estimate : [1.0433808  0.39344854] 
 		[0.39344854 1.69759236] 	 Error : 0.6347722240374717 

N100 estimate : [1.09796675 0.06568148] 
 		[0.06568148 1.43287195] 	 Error : 0.5829749793964033 

N1000 estimate : [ 1.00411284 -0.00912175] 
 		[-0.00912175  1.92353726] 	 Error : 0.07765229561546955 



### Ques - 3 : Bayesian Est with Non-informative Prior

In [ ]:
def bayesian_estimate_jeffery(samples):
    # https://tminka.github.io/papers/minka-gaussian.pdf
    n = len(samples)
    d = 1
    dof = n
    scale = samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

def bayesian_estimate_ind_jeffery(samples):
    # gelman page 72
    n = len(samples)
    d = 1
    dof = n - 1
    scale = samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

In [ ]:
jeffery_n10 = bayesian_estimate_jeffery(n10)
jeffery_n100 = bayesian_estimate_jeffery(n100)
jeffery_n1000 = bayesian_estimate_jeffery(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {jeffery_n10[0]} \n \t\t{jeffery_n10[1]} \t", f"Error : {get_error(true_cov, jeffery_n10)}" , "\n")
print(f"N100 estimate : {jeffery_n100[0]} \n \t\t{jeffery_n100[1]} \t", f"Error : {get_error(true_cov, jeffery_n100)}", "\n")
print(f"N1000 estimate : {jeffery_n1000[0]} \n \t\t{jeffery_n1000[1]} \t", f"Error : {get_error(true_cov, jeffery_n1000)}" , "\n") 
print("\n")

jeffery_ind_n10 = bayesian_estimate_ind_jeffery(n10)
jeffery_ind_n100 = bayesian_estimate_ind_jeffery(n100)
jeffery_ind_n1000 = bayesian_estimate_ind_jeffery(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {jeffery_ind_n10[0]} \n \t\t{jeffery_ind_n10[1]} \t", f"Error : {get_error(true_cov, jeffery_ind_n10)}" , "\n")
print(f"N100 estimate : {jeffery_ind_n100[0]} \n \t\t{jeffery_ind_n100[1]} \t", f"Error : {get_error(true_cov, jeffery_ind_n100)}", "\n")
print(f"N1000 estimate : {jeffery_ind_n1000[0]} \n \t\t{jeffery_ind_n1000[1]} \t", f"Error : {get_error(true_cov, jeffery_ind_n1000)}" , "\n")  

N10 estimate : [1.1954938  0.63935387] 
 		[0.63935387 2.13358758] 	 Error : 0.9346711809305068 

N100 estimate : [1.11316913 0.06903258] 
 		[0.06903258 1.45495725] 	 Error : 0.5651635564589236 

N1000 estimate : [ 1.00513545 -0.00916745] 
 		[-0.00916745  1.9281642 ] 	 Error : 0.07317676920946625 



N10 estimate : [1.36627863 0.73069014] 
 		[0.73069014 2.43838581] 	 Error : 1.1807448136260692 

N100 estimate : [1.1246451  0.06974426] 
 		[0.06974426 1.46995681] 	 Error : 0.5533630885390415 

N1000 estimate : [ 1.00614361 -0.00917664] 
 		[-0.00917664  1.93009816] 	 Error : 0.0713612795160005 



### Ques 4 : MC Bayesian Est

In [ ]:
def mc_bayesian_a(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[4,0],[0,5]])
    vals = []

    for m in [1e3, 1e4, 1e5]:
        numerators = []
        denominators = []
        for _ in range(int(m)):
            sigma_j = invwishart.rvs(v0,delta0)


            det_term = np.linalg.det(sigma_j)**(-n/2)
            det_term = np.clip(det_term, 1e-100, 1e100)

            exp_term = np.exp(np.clip(-0.5*np.trace(np.linalg.inv(sigma_j)@(samples.T@samples)), -100, 100))
            numerators.append(sigma_j*det_term*exp_term)
            denominators.append(det_term*exp_term)

        print(f"For m={m}:")
        vals.append(np.array(numerators).mean(axis=0)/np.mean(denominators))
        print(np.array(numerators).mean(axis=0)/np.mean(denominators))
        print("\n")
    
    return np.array(vals)

def mc_bayesian_b(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[2,0],[0,4]])
    vals = []

    for m in [1e3, 1e4, 1e5]:
        numerators = []
        denominators = []
        for _ in range(int(m)):
            sigma_j = invwishart.rvs(v0,delta0)
            
            det_term = np.linalg.det(sigma_j)**(-n/2)
            det_term = np.clip(det_term, 1e-100, 1e100)

            exp_term = np.exp(np.clip(-0.5*np.trace(np.linalg.inv(sigma_j)@(samples.T@samples)), -100, 100))
            numerators.append(sigma_j*det_term*exp_term)
            denominators.append(det_term*exp_term)

        print(f"For m={m}:")
        vals.append(np.array(numerators).mean(axis=0)/np.mean(denominators))
        print(np.array(numerators).mean(axis=0)/np.mean(denominators))
        print("\n")
    
    return np.array(vals)


In [ ]:
mc_a_n10 = mc_bayesian_a(n10)
mc_a_n100 = mc_bayesian_a(n100)
mc_a_n1000 = mc_bayesian_a(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {mc_a_n10[0]} \n \t\t{mc_a_n10[1]} \t", f"Error : {[get_error(true_cov, x) for x in mc_a_n10] }" , "\n")
print(f"N100 estimate : {mc_a_n100[0]} \n \t\t{mc_a_n100[1]} \t", f"Error : {[get_error(true_cov, x) for x in mc_a_n100]}", "\n")
print(f"N1000 estimate : {mc_a_n1000[0]} \n \t\t{mc_a_n1000[1]} \t", f"Error : {[get_error(true_cov, x) for x in mc_a_n1000]}" , "\n") 
print("\n")

mc_b_n10 =  mc_bayesian_b(n10)
mc_b_n100 =  mc_bayesian_b(n100)
mc_b_n1000 =  mc_bayesian_b(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {mc_b_n10[0]} \n \t\t{mc_b_n10[1]} \t", f"Error : {[get_error(true_cov, x) for x in mc_b_n10]}" , "\n")
print(f"N100 estimate : {mc_b_n100[0]} \n \t\t{mc_b_n100[1]} \t", f"Error : {[get_error(true_cov, x) for x in mc_b_n100]}", "\n")
print(f"N1000 estimate : {mc_b_n1000[0]} \n \t\t{mc_b_n1000[1]} \t", f"Error : {[get_error(true_cov, x) for x in mc_b_n1000]}" , "\n")  

For m=1000.0:
[[1.14750261 0.41344169]
 [0.41344169 1.76454058]]


For m=10000.0:
[[1.13729556 0.43422316]
 [0.43422316 1.84828843]]


For m=100000.0:
[[1.12964945 0.42687344]
 [0.42687344 1.83704482]]


For m=1000.0:
[[ 0.31320765 -0.05271226]
 [-0.05271226  0.31963124]]


For m=10000.0:
[[ 0.28152285 -0.03961383]
 [-0.03961383  0.26570593]]


For m=100000.0:
[[ 0.21161226 -0.06975444]
 [-0.06975444  0.27004901]]




C:\Users\pcdpm\AppData\Local\Temp\ipykernel_20396\2857128785.py:19: RuntimeWarning: overflow encountered in scalar power
  det_term = np.linalg.det(sigma_j)**(-n/2)


For m=1000.0:
[[0.64683946 0.02515833]
 [0.02515833 0.82488662]]


For m=10000.0:
[[0.65887673 0.00255673]
 [0.00255673 0.82126412]]


For m=100000.0:
[[ 6.59941724e-01 -8.11756238e-04]
 [-8.11756238e-04  8.25364219e-01]]


N10 estimate : [[1.14750261 0.41344169]
 [0.41344169 1.76454058]] 
 		[[1.13729556 0.43422316]
 [0.43422316 1.84828843]] 	 Error : [0.6473532371247749, 0.6472758192414859, 0.6385963029495249] 

N100 estimate : [[ 0.31320765 -0.05271226]
 [-0.05271226  0.31963124]] 
 		[[ 0.28152285 -0.03961383]
 [-0.03961383  0.26570593]] 	 Error : [1.8168324312854691, 1.8780638531030667, 1.9036851112569915] 

N1000 estimate : [[0.64683946 0.02515833]
 [0.02515833 0.82488662]] 
 		[[0.65887673 0.00255673]
 [0.00255673 0.82126412]] 	 Error : [1.227550285214802, 1.2271089734121554, 1.2228696443345275] 



For m=1000.0:
[[0.97055226 0.44532681]
 [0.44532681 1.78088291]]


For m=10000.0:
[[0.97500692 0.43525919]
 [0.43525919 1.76810838]]


For m=100000.0:
[[0.96971216 0.43069749]
 [0.43

C:\Users\pcdpm\AppData\Local\Temp\ipykernel_20396\2857128785.py:45: RuntimeWarning: overflow encountered in scalar power
  det_term = np.linalg.det(sigma_j)**(-n/2)


For m=1000.0:
[[0.46848878 0.00661777]
 [0.00661777 0.98335474]]


For m=10000.0:
[[ 0.47172908 -0.00914788]
 [-0.00914788  0.95894768]]


For m=100000.0:
[[ 0.47680919 -0.00103238]
 [-0.00103238  0.95771684]]


N10 estimate : [[0.97055226 0.44532681]
 [0.44532681 1.78088291]] 
 		[[0.97500692 0.43525919]
 [0.43525919 1.76810838]] 	 Error : [0.6674664118655235, 0.6582548852753701, 0.6565192995883354] 

N100 estimate : [[ 0.20484956 -0.00955182]
 [-0.00955182  0.29402739]] 
 		[[0.09728612 0.04216625]
 [0.04216625 0.26260529]] 	 Error : [1.882229855168189, 1.9588233016940195, 1.9522045779222439] 

N1000 estimate : [[0.46848878 0.00661777]
 [0.00661777 0.98335474]] 
 		[[ 0.47172908 -0.00914788]
 [-0.00914788  0.95894768]] 	 Error : [1.1472398805591821, 1.167487675616894, 1.1662267964403852] 



### Ques - 5 : Gibbs sampling

In [ ]:
def gibbs_sampler(samples):
    n = len(samples)
    d = 2
    v = 5 # to be checked
    iter = 1e5
    A_1 = 0.05
    A_2 = 0.05

    a1 = invgamma.rvs(0.5, scale = 1/(A_1**2))
    a2 = invgamma.rvs(0.5, scale = 1/(A_1**2))
    
    sigmas = []
    for i in tqdm(range(int(iter))):
        if i%2 == 0:
            sigma = invwishart.rvs(v+d+n-1, 2*v*np.array([[1/a1,0],[0,1/a2]])+samples.T@samples)
            sigmas.append(sigma)
        else:
            a1 = invgamma.rvs((v+n)/2,scale=v*(np.linalg.inv(sigma)[0][0])+1/(A_1**2))
            a2 = invgamma.rvs((v+n)/2,scale=v*(np.linalg.inv(sigma)[1][1])+1/(A_2**2))
    
    return np.array(sigmas[-1000:]).mean(axis=0)

In [ ]:
gibbs_n10 = gibbs_sampler(n10)
gibbs_n100 = gibbs_sampler(n100)
gibbs_n1000 = gibbs_sampler(n1000)

# print(f"  Estimate  \t|\t Error \n")
print(f"N10 estimate : {gibbs_n10[0]} \n \t\t{gibbs_n10[1]} \t", f"Error : {get_error(true_cov, gibbs_n10)}" , "\n")
print(f"N100 estimate : {gibbs_n100[0]} \n \t\t{gibbs_n100[1]} \t", f"Error : {get_error(true_cov, gibbs_n100)}", "\n")
print(f"N1000 estimate : {gibbs_n1000[0]} \n \t\t{gibbs_n1000[1]} \t", f"Error : {get_error(true_cov, gibbs_n1000)}" , "\n") 
print("\n")

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

N10 estimate : [0.74187307 0.3909514 ] 
 		[0.3909514  1.34889428] 	 Error : 0.89233074886173 

N100 estimate : [1.06740812 0.06234312] 
 		[0.06234312 1.40153267] 	 Error : 0.608670949224485 

N1000 estimate : [ 1.01154891 -0.01157512] 
 		[-0.01157512  1.93106917] 	 Error : 0.07178303140038926 





### Ques 6 : Empirical Bayes

In [43]:
def EmpiricalBayes(samples,v0):
    n,d = samples.shape
    v_opt = minimize(lambda x: objective_function_voptimal(x,n,d), v0)
    del_opt = v_opt.x[0]/n * samples.T@samples 
    #  InvWishart(νopt + n,∆opt + Σn) Calculate the posterior mean
    posterior_mean = (del_opt + samples.T@samples ) / (v_opt.x[0] + n - d - 1)
    return posterior_mean


In [45]:
print(f"For n = 10 \n{EmpiricalBayes(n10,5)}")
print(f"For n = 10 \n{EmpiricalBayes(n100,5)}")
print(f"For n = 10 \n{EmpiricalBayes(n1000,5)}")

For n = 10 
[[0.63347098 0.19132897]
 [0.19132897 1.32014929]]
For n = 10 
[[ 1.12432537 -0.00780099]
 [-0.00780099  2.3160673 ]]
For n = 10 
[[1.04545965 0.01202806]
 [0.01202806 2.01267675]]


C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py:4: RuntimeWarning: overflow encountered in multiply
  init *= gamma(x + (1-i)/2)
C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py:9: RuntimeWarning: invalid value encountered in divide
  term_3 = np.log(gammafn(v/2,d)/gammafn((v+n)/2,d))
C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py:4: RuntimeWarning: overflow encountered in multiply
  init *= gamma(x + (1-i)/2)
C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py:9: RuntimeWarning: invalid value encountered in divide
  term_3 = np.log(gammafn(v/2,d)/gammafn((v+n)/2,d))
C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py:9: RuntimeWarning: invalid value encountered in divide
  term_3 = np.log(gammafn(v/2,d)/gammafn((v+n)/2,d))
C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py:4: RuntimeWarning: overflow encountered in multiply
  init *= gamma(x + (1-i)/2)
C:\Users\aksha\AppData\Local\Temp\ipykernel_22688\50817309.py

## Code without outputs

In [11]:
np.random.seed(42)

In [12]:
def generate_samples(n):
    # size = nx2
    mean = np.array([0,0])
    cov = np.array([[1,0],[0,2]])

    samples = np.random.multivariate_normal(mean,cov,size=n)
    return samples

In [13]:
n10 = generate_samples(10)
n100 = generate_samples(100)
n1000 = generate_samples(1000)

### Ques - 1 : MLE

In [14]:
def mle_estimate(samples):
    return samples.T@samples/len(samples)

### Ques - 2 : Bayesian Est with Conjugate Prior

In [ ]:
def bayesian_estimate_invwishart_cp(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[4,0],[0,5]])
    d = 1
    dof = n + v0
    scale = delta0 + samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

### Ques - 3 : Bayesian Est with Non-informative Prior

In [ ]:
def bayesian_estimate_jeffery(samples):
    # https://tminka.github.io/papers/minka-gaussian.pdf
    n = len(samples)
    d = 1
    dof = n
    scale = samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

def bayesian_estimate_ind_jeffery(samples):
    # gelman page 72
    n = len(samples)
    d = 1
    dof = n - 1
    scale = samples.T@samples
    
    # return analytical mean of posterior
    return scale/(dof-d-1)

### Ques 4 : MC Bayesian Est

In [75]:
def mc_bayesian_a(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[4,0],[0,5]])

    for m in [1e3, 1e4, 1e5]:
        numerators = []
        denominators = []
        for _ in range(int(m)):
            sigma_j = invwishart.rvs(v0,delta0)
            det_term = np.linalg.det(sigma_j)**(-n/2)
            exp_term = np.exp(-0.5*np.linalg.trace(np.linalg.inv(sigma_j)@(samples.T@samples)))
            numerators.append(sigma_j*det_term*exp_term)
            denominators.append(det_term*exp_term)

        print(f"For m={m}:\n")
        print(np.array(numerators).mean(axis=0)/np.mean(denominators))

def mc_bayesian_b(samples):
    n = len(samples)
    v0 = 5
    delta0 = np.array([[2,0],[0,4]])

    for m in [1e3, 1e4, 1e5]:
        numerators = []
        denominators = []
        for _ in range(int(m)):
            sigma_j = invwishart.rvs(v0,delta0)
            det_term = np.linalg.det(sigma_j)**(-n/2)
            exp_term = np.exp(-0.5*np.linalg.trace(np.linalg.inv(sigma_j)@(samples.T@samples)))
            numerators.append(sigma_j*det_term*exp_term)
            denominators.append(det_term*exp_term)

        print(f"For m={m}:\n")
        print(np.array(numerators).mean(axis=0)/np.mean(denominators))


### Ques - 5 : Gibbs sampling

In [78]:
def gibbs_sampler(samples):
    n = len(samples)
    d = 2
    v = 5 # to be checked
    iter = 1e3
    A_1 = 0.05
    A_2 = 0.05

    a1 = invgamma.rvs(0.5, scale = 1/(A_1**2))
    a2 = invgamma.rvs(0.5, scale = 1/(A_1**2))
    
    sigmas = []
    for i in range(int(iter)):
        if i%2 == 0:
            sigma = invwishart.rvs(v+d+n-1, 2*v*np.array([[1/a1,0],[0,1/a2]])+samples.T@samples)
            sigmas.append(sigma)
        else:
            a1 = invgamma.rvs((v+n)/2,scale=v*(np.linalg.inv(sigma)[0][0])+1/(A_1**2))
            a2 = invgamma.rvs((v+n)/2,scale=v*(np.linalg.inv(sigma)[1][1])+1/(A_2**2))
    
    return np.array(sigmas).mean(axis=0)
            
        

### Ques 6 : Empirical Bayes